In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain import hub

from datetime import datetime

from dotenv import load_dotenv
import os
import math
load_dotenv()


True

In [ ]:
# 모델 초기화
chat_model = ChatOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model_name="gpt-3.5-turbo-1106", temperature=0.25)

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("resource/ai_trand.pdf")

# 불러온 PDF 파일의 내용 분할을 위한 splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=256)

# 페이지 별 문서 로드
docs = loader.load()
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 벡터스토어를 효율적인 검색을 위해 retriever로 변환
retriever = vectorstore.as_retriever()
# Hub로 부터 Prompt 받아오기
prompt = hub.pull("rlm/rag-prompt")

ValueError: File path docs/ai_trand.pdf is not a valid file or url

In [ ]:
# 문서 포맷팅을 위함 함수 2개의 줄바꿈을 삽입해 문자열로 결합
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG 체인 정의
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser()
)


In [ ]:
# 질문에 대한 대답을 한줄씩 나눠서 저장시키기 위한 함수 정의
def split_long_text(text, line_length=80):
    return [text[i:i+line_length] for i in range(0, len(text), line_length)]

now_dir = os.getcwd()

# prompt 폴더가 있는지 확인
if os.path.isdir(now_dir + "\Prompts"):
    # prompt 폴더에 파일이 있는지 확인
    prompt_file = os.listdir(now_dir + "\Prompts")
    if len(prompt_file) == 0:
        print("Prompt폴더에 프롬프트 txt파일을 생성해주세요")
        
    for file in prompt_file:
        filename_split = os.path.splitext(now_dir+"\Prompts\\" + file)
        if filename_split[1] == ".txt":
            # prompt 내용 읽기
            f = open(now_dir+"\Prompts\\" + file, 'r', encoding='UTF8') 
            lines = f.readlines()
            # RAG 체인 실행
            response = rag_chain.invoke(lines[0])
            
            file_split = os.path.basename(file).split(".")
            now = datetime.now()
            
            # 실행에 대한 결과 파일 경로
            file_dir = now_dir + "\Result\\"
            file_name = file_split[0] + "_result_" + str(math.floor(now.timestamp())) + filename_split[1]
            w = open(file_dir + file_name, 'w', encoding='UTF8') 
            for line in split_long_text(response, line_length=80):            
                w.write(line + '\n')
            w.close()
else:
    print("Prompt폴더를 생성해서 프롬프트 txt파일을 생성해주세요")